# Process SmolLM Corpus

In [ ]:
from datasets import load_dataset
import boto3
import gzip,os

# cv2 = load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2")
# fed = load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup")
# pe = load_dataset("HuggingFaceTB/smollm-corpus", "python-edu")

# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed4 = fed.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=120)
# fed4=load_dataset("chengjunyan1/smollm-10", "fineweb-edu-dedup")

In [ ]:
pe4 = pe.filter(lambda x: x['score'] > 4.16, num_proc=120)
pe4['train'].num_rows,pe4['train'].num_rows/pe['train'].num_rows

In [ ]:
from datasets import dataset_dict

cv2_ratio=(fed4['train'].num_rows/fed['train'].num_rows+pe4['train'].num_rows/pe['train'].num_rows)/2
cv2_samples=int(cv2.num_rows['train']*cv2_ratio)
cv2_10=cv2['train'].shuffle(seed=42).select(range(cv2_samples))
cv2_10=dataset_dict.DatasetDict({'train':cv2_10})

In [ ]:
len_all=fed4['train'].num_rows+pe4['train'].num_rows+cv2_10['train'].num_rows
len_all_full=fed['train'].num_rows+pe['train'].num_rows+cv2['train'].num_rows
print(fed4['train'].num_rows/len_all, pe4['train'].num_rows/len_all, cv2_10['train'].num_rows/len_all)
print(fed['train'].num_rows/len_all_full, pe['train'].num_rows/len_all_full, cv2['train'].num_rows/len_all_full)

In [ ]:
from huggingface_hub import login

login()

In [ ]:
# pe4.push_to_hub("chengjunyan1/smollm-10","python-edu")
# fed4.push_to_hub("chengjunyan1/smollm-10","fineweb-edu-dedup")
# cv2_10.push_to_hub("chengjunyan1/smollm-10","cosmopedia-v2")

# Verify processed corpus

In [ ]:
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
import functools as ft


# os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"
# os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"

In [ ]:
fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup")
cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2")

In [ ]:
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

pe10=download_python_edu()

In [ ]:
from datasets import concatenate_datasets,DatasetDict
import numpy as np


def resample_dataset(dataset, weight):
    num_samples = int(len(dataset) * weight)
    indices = np.random.choice(len(dataset), num_samples, replace=True)
    return dataset.select(indices)

def combine_datasets(dataset_dicts, weights:dict=None): # weights e.g. {'train':[1.5,1.0]}
    combined_dict = {}
    
    # Initialize weights if not provided
    for dataset_dict in dataset_dicts:
        for key, dataset in dataset_dict.items():
            if key in combined_dict:
                combined_dict[key] = concatenate_datasets([combined_dict[key], dataset])
            else:
                combined_dict[key] = dataset

    # Apply weights by resampling the datasets
    for key in combined_dict:
        datasets = []
        for idx,dataset in enumerate(dataset_dicts):
            if key in dataset:
                if weights is None or key not in weights or weights[key][idx] == 1.0:
                    datasets.append(dataset[key])
                else:
                    resampled_dataset = resample_dataset(dataset[key], weights[key][idx])
                    datasets.append(resampled_dataset)
        combined_dict[key] = concatenate_datasets(datasets)
    
    return DatasetDict(combined_dict)


dataset_dicts=[pe10,fed10,cv10]
combined=combine_datasets(dataset_dicts)

# Create test split based on the processed corpus

In [ ]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login


# login('YOURKEY')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"
os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

In [ ]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)
# fed10=load_dataset("chengjunyan1/smollm-10","fineweb-edu-dedup",num_proc=DEFAULT_NUM_PROC//8)
# cv10=load_dataset("chengjunyan1/smollm-10","cosmopedia-v2",num_proc=DEFAULT_NUM_PROC//8)
# pe10=download_python_edu()

In [ ]:
# random sample 2 GB of data from each dataset
smollm_size=673
test_ratio=1/smollm_size
pe_test_lines=int(pe['train'].num_rows*test_ratio)
fed_test_lines=int(fed['train'].num_rows*test_ratio)
cv_test_lines=int(cv['train'].num_rows*test_ratio)
pe_shuffle=pe['train'].shuffle(seed=42)
fed_shuffle=fed['train'].shuffle(seed=42)
cv_shuffle=cv['train'].shuffle(seed=42)
pe_test=pe_shuffle.select(range(pe_test_lines))
pe_eval=pe_shuffle.select(range(pe_test_lines,pe_test_lines*2))
pe_test=pe_test.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
pe_eval=pe_eval.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
fed_test=fed_shuffle.select(range(fed_test_lines))
fed_eval=fed_shuffle.select(range(fed_test_lines,fed_test_lines*2))
cv_test=cv_shuffle.select(range(cv_test_lines))
cv_eval=cv_shuffle.select(range(cv_test_lines,cv_test_lines*2))

print('Python-Edu:',pe10.num_rows,pe_eval.num_rows,pe_test.num_rows)
print('FineWeb-Edu-Dedup:',fed10.num_rows,fed_eval.num_rows,fed_test.num_rows)
print('Cosmopedia-V2:',cv10.num_rows,cv_eval.num_rows,cv_test.num_rows)

In [ ]:
pe_test,pe_eval,fed_test,fed_eval,cv_test,cv_eval

In [ ]:
# remove verbatims
import multiprocessing as mp
def get_text(x):
    return x['text']
with mp.Pool(mp.cpu_count()) as pool:
    pe_text=pool.map(get_text, pe_test)+pool.map(get_text, pe_eval)
    fed_text=pool.map(get_text, fed_test)+pool.map(get_text, fed_eval)
    cv_text=pool.map(get_text, cv_test)+pool.map(get_text, cv_eval)

In [ ]:
pe_train=pe10.filter(lambda x: x['text'] not in pe_text, num_proc=DEFAULT_NUM_PROC//4-8)
cv_train=cv10.filter(lambda x: x['text'] not in cv_text, num_proc=DEFAULT_NUM_PROC//4-8)
fed_train=fed10.filter(lambda x: x['text'] not in fed_text, num_proc=DEFAULT_NUM_PROC//4-8)

In [ ]:
print(pe_train['train'].num_rows,pe10['train'].num_rows,pe_train['train'].num_rows/pe10['train'].num_rows)
print(cv_train['train'].num_rows,cv10['train'].num_rows,cv_train['train'].num_rows/cv10['train'].num_rows)
print(fed_train['train'].num_rows,fed10['train'].num_rows,fed_train['train'].num_rows/fed10['train'].num_rows)
total_rows_train=pe_train['train'].num_rows+cv_train['train'].num_rows+fed_train['train'].num_rows
total_rows=pe10['train'].num_rows+cv10['train'].num_rows+fed10['train'].num_rows
print(pe_train['train'].num_rows/total_rows_train,cv_train['train'].num_rows/total_rows_train,fed_train['train'].num_rows/total_rows_train)
print(pe10['train'].num_rows/total_rows,cv10['train'].num_rows/total_rows,fed10['train'].num_rows/total_rows)

In [ ]:
pe10_corpus=DatasetDict({'train':pe10['train'],'eval':pe_eval,'test':pe_test})
fed10_corpus=DatasetDict({'train':fed10['train'],'eval':fed_eval,'test':fed_test})
cv10_corpus=DatasetDict({'train':cv10['train'],'eval':cv_eval,'test':cv_test})

In [ ]:
pe10_corpus,fed10_corpus,cv10_corpus

In [ ]:
# pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")
# fed10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
# cv10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

In [ ]:
# test the datasets
from datasets import load_dataset
pe10_corpus=load_dataset("chengjunyan1/smollm-10-corpus","python-edu")
fed10_corpus=load_dataset("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
cv10_corpus=load_dataset("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

In [ ]:
pe10_corpus,fed10_corpus,cv10_corpus

In [ ]:
def download_python_edu(split):
    ds = load_dataset("chengjunyan1/smollm-10-corpus", "python-edu", split=split, num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

pe10_train=download_python_edu("train")
pe10_eval=download_python_edu("eval")
pe10_test=download_python_edu("test")
pe10_corpus=DatasetDict({'train':pe10_train['train'],'eval':pe10_eval['train'],'test':pe10_test['train']})

In [ ]:
# pe10_corpus,fed10_corpus,cv10_corpus
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')


splits = ['train','eval','test']

# for split in splits:
#     print('Counting',split)
#     fed10_token_lengths=np.sum(fed10_corpus[split]['metadata']['token_count'])
#     print(f'{fed10_token_lengths/1000/1000/1000:.2f}','B')

# for split in splits:
#     print('Counting',split)
#     cv10_token_lengths=np.sum(cv10_corpus[split]['token_length'])
#     print(f'{cv10_token_lengths/1000/1000/1000:.2f}','B')

for split in splits:
    print('Counting',split)
    pe10_length=pe10_corpus[split].map(lambda x: {'token_length':tokenizer(x['text'],return_tensors='pt')['input_ids'].shape[1]},num_proc=cpu_count())
    pe10_token_lengths=np.sum(pe10_length['token_length'])
    print(f'{pe10_token_lengths/1000/1000/1000:.3f}','B')


In [ ]:
#remove text column from pe10

# pe10_corpus['train']=pe10_corpus['train'].remove_columns('text')
# pe10_corpus['eval']=pe10_corpus['eval'].remove_columns('text')
# pe10_corpus['test']=pe10_corpus['test'].remove_columns('text')

# pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")

# Make train and test splits from scratch

In [ ]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login
DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YOURKEY"
os.environ["AWS_SECRET_ACCESS_KEY"]="YOURKEY"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu():
    ds = load_dataset("chengjunyan1/smollm-10", "python-edu", split="train", num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({"train": ds})
    return ds

In [ ]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)

In [ ]:
# random sample 2 GB of data from each dataset
smollm_size=673
test_ratio=1/smollm_size
pe_test_lines=int(pe['train'].num_rows*test_ratio)
fed_test_lines=int(fed['train'].num_rows*test_ratio)
cv_test_lines=int(cv['train'].num_rows*test_ratio)
pe_shuffle=pe['train'].shuffle(seed=42)
fed_shuffle=fed['train'].shuffle(seed=42)
cv_shuffle=cv['train'].shuffle(seed=42)
pe_test=pe_shuffle.select(range(pe_test_lines))
pe_eval=pe_shuffle.select(range(pe_test_lines,pe_test_lines*2))
pe_train=pe_shuffle.select(range(pe_test_lines*2,pe_shuffle.num_rows))
fed_test=fed_shuffle.select(range(fed_test_lines))
fed_eval=fed_shuffle.select(range(fed_test_lines,fed_test_lines*2))
fed_train=fed_shuffle.select(range(fed_test_lines*2,fed_shuffle.num_rows))
cv_test=cv_shuffle.select(range(cv_test_lines))
cv_eval=cv_shuffle.select(range(cv_test_lines,cv_test_lines*2))
cv_train=cv_shuffle.select(range(cv_test_lines*2,cv_shuffle.num_rows))

print('Python-Edu:',pe_train.num_rows,pe_eval.num_rows,pe_test.num_rows)
print('FineWeb-Edu-Dedup:',fed_train.num_rows,fed_eval.num_rows,fed_test.num_rows)
print('Cosmopedia-V2:',cv_train.num_rows,cv_eval.num_rows,cv_test.num_rows)

In [ ]:
pe10 = pe_train.filter(lambda x: x['score'] > 4.16, num_proc=DEFAULT_NUM_PROC//4-8)
fed10 = fed_train.filter(lambda x: x['metadata']['int_score'] >= 4, num_proc=DEFAULT_NUM_PROC//4-8)
cv2_ratio=(fed10['train'].num_rows/fed_train['train'].num_rows+pe10['train'].num_rows/pe_train['train'].num_rows)/2
cv2_samples=int(cv_train.num_rows['train']*cv2_ratio)
cv10=cv_train['train'].shuffle(seed=42).select(range(cv2_samples))
cv10=dataset_dict.DatasetDict({'train':cv10})

In [ ]:
pe10_corpus=DatasetDict({'train':pe10['train'],'eval':pe_eval,'test':pe_test})
fed10_corpus=DatasetDict({'train':fed10['train'],'eval':fed_eval,'test':fed_test})
cv10_corpus=DatasetDict({'train':cv10['train'],'eval':cv_eval,'test':cv_test})

In [ ]:
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","python-edu")
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","fineweb-edu-dedup")
pe10_corpus.push_to_hub("chengjunyan1/smollm-10-corpus","cosmopedia-v2")

# V2 Make based on SmolLM Mixture 

In [ ]:
import os
from datasets import load_dataset,DatasetDict
import boto3
import gzip,os
from huggingface_hub import login
import numpy as np

# login('YOURKEY')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

os.environ["AWS_ACCESS_KEY_ID"]="YK"
os.environ["AWS_SECRET_ACCESS_KEY"]="YK"

session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")
bucket_name = "softwareheritage"

def download_contents(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket=bucket_name, Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}

def download_python_edu(path,split):
    ds = load_dataset(path, "python-edu", split=split, num_proc=DEFAULT_NUM_PROC)
    ds = ds.map(download_contents, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC)
    ds = DatasetDict({split: ds})
    return ds

In [ ]:
pe=load_dataset("HuggingFaceTB/smollm-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC//8)
fed=load_dataset("HuggingFaceTB/smollm-corpus", "fineweb-edu-dedup", num_proc=DEFAULT_NUM_PROC//8)
cv=load_dataset("HuggingFaceTB/smollm-corpus", "cosmopedia-v2", num_proc=DEFAULT_NUM_PROC//8)

In [ ]:
fed4 = fed.filter(lambda x: x['metadata']['score'] >= 4, num_proc=120)

In [ ]:
fed4

In [ ]:
from tqdm import tqdm

fed4_token_lengths=np.sum(fed4['train']['metadata']['token_count'])
# bar=tqdm(fed4['train'])
# for i in bar:
#     fed4_token_lengths+=i['metadata']['token_count']
#     current_count=f'{fed4_token_lengths/1000/1000/1000:.3f}B'
#     bar.set_description(current_count)

## Process PY

In [ ]:
from datasets import load_dataset
import os


session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])
s3 = session.client("s3")

def download_contents_py(blob_id):
    key = f"content/{blob_id}"
    obj = s3.get_object(Bucket="softwareheritage", Key=key)
    with gzip.GzipFile(fileobj=obj['Body']) as fin:
        content = fin.read().decode("utf-8", errors="ignore")
    return {"text": content}


DEFAULT_NUM_PROC_LOAD=os.cpu_count()*4

ds = load_dataset("chengjunyan1/smollm-12.5-corpus", "python-edu", num_proc=DEFAULT_NUM_PROC_LOAD)

In [ ]:
DEFAULT_NUM_PROC_LOAD=os.cpu_count()*8
from datasets import DatasetDict    

ds_train = ds['train'].map(download_contents_py, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC_LOAD)
ds_test = ds['test'].map(download_contents_py, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC_LOAD)
ds_eval = ds['eval'].map(download_contents_py, input_columns="blob_id", num_proc=DEFAULT_NUM_PROC_LOAD)
ds = DatasetDict({'train':ds_train, 'test':ds_test, 'eval':ds_eval})

In [ ]:

# ds.push_to_hub("chengjunyan1/smollm-12.5-corpus","python-edu")


## Process DeepMind Math

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

import os
from huggingface_hub import login
import numpy as np

# login('')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download


In [ ]:
ds = load_dataset("chengjunyan1/smollm-12.5-corpus", "deepmind-math", num_proc=DEFAULT_NUM_PROC)


In [ ]:
ds

In [ ]:
#shuffle
ds['train']=ds['train'].shuffle(seed=42)
ds['test']=ds['test'].shuffle(seed=42)

In [ ]:
ds['test']

In [ ]:
# around 195,167,286
rows=50000
print(round(sum(ds['test'][:rows]['token_counts'])/1000/1000/1000,5))



In [ ]:
ds

In [ ]:
ds_new=DatasetDict({
    'train':ds['train'].select(range(3880000)),
    'test':ds['test'].select(range(rows)),
    'eval':ds['test'].select(range(rows,rows*2))
})
ds_new


In [ ]:
ds_new.push_to_hub("chengjunyan1/smollm-12.5-corpus","deepmind-math-small")

## Process StackOverflow

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

import os
from huggingface_hub import login
import numpy as np

# login('')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download


ds = load_dataset("bigcode/stackoverflow-clean",num_proc=DEFAULT_NUM_PROC)

In [ ]:
ds=ds.shuffle(seed=42)


In [ ]:
n_rows_train=395000

ds_train=ds['train'].select(range(n_rows_train))
nb_train=round(sum(ds_train['nb_tokens'])/1000/1000/1000,5)
print(nb_train)

In [ ]:
n_rows_test=6100
n_rows_eval=n_rows_test

ds_test=ds['train'].select(range(n_rows_train,n_rows_train+n_rows_test))
nb_test=round(sum(ds_test['nb_tokens'])/1000/1000/1000,5)
print(nb_test)
ds_eval=ds['train'].select(range(n_rows_train+n_rows_test,n_rows_train+n_rows_test+n_rows_eval))
nb_eval=round(sum(ds_eval['nb_tokens'])/1000/1000/1000,5)
print(nb_eval)

print(nb_train+nb_test+nb_eval)

In [ ]:
ds_new=DatasetDict({
    'train':ds_train,
    'test':ds_test,
    'eval':ds_eval
})
ds_new.push_to_hub("chengjunyan1/smollm-12.5-corpus","stackoverflow-clean")

## Diagnose Cosmopedia 

In [ ]:
from datasets import load_dataset
from datasets import DatasetDict

import os
from huggingface_hub import login
import numpy as np

# login('')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download


ds = load_dataset("chengjunyan1/smollm-12.5-corpus","cosmopedia-v2",num_proc=DEFAULT_NUM_PROC)


In [ ]:
ds_text=ds['train'].map(lambda x: {'text':f"Audience: {x['audience']}\n\nPrompt: {x['prompt']}\n\nResponse: {x['text']}"},num_proc=DEFAULT_NUM_PROC)

# Test loading all datasets

In [ ]:
import sys
sys.path.append('..')

from model_discovery.configs.gam_config import ( 
    GAMConfig,GAMConfig_14M,GAMConfig_31M,GAMConfig_70M,GAMConfig_125M,GAMConfig_350M,GAMConfig_760M,
    GAMConfig_1300M,GAMConfig_2700M,GAMConfig_6700M,GAMConfig_13B,GAMConfig_175B,GAMConfig_1T,GAMConfig_debug
)

from model_discovery.ve.data_loader import load_datasets


cfg=GAMConfig_14M()

# Process test data

In [3]:
from datasets import load_dataset
from datasets import DatasetDict

import os
from huggingface_hub import login
import numpy as np

# login('')

DEFAULT_NUM_PROC =  os.cpu_count()*4 # Configure it based on your system, it can significantly speed up the download

def process_cosmopedia_v2(examples):
    return {
        'text': [
            f"Audience: {audience}\n\nPrompt: {prompt}\n\nResponse: {text}"
            for audience, prompt, text in zip(examples['audience'], examples['prompt'], examples['text'])
        ]
    }

def process_deepmind_math_small(examples):
    return {
        'text': [
            f"Question: {question}\n\nAnswer: {answer}"
            for question, answer in zip(examples['question'], examples['answer'])
        ]
    }



python_edu=load_dataset("chengjunyan1/smollm-12.5-corpus","python-edu",split='test',num_proc=DEFAULT_NUM_PROC)
fineweb_edu_dedup=load_dataset("chengjunyan1/smollm-12.5-corpus","fineweb-edu-dedup",split='test',num_proc=DEFAULT_NUM_PROC)
open_web_math=load_dataset("chengjunyan1/smollm-12.5-corpus","open-web-math",split='test',num_proc=DEFAULT_NUM_PROC)
cosmopedia = load_dataset("chengjunyan1/smollm-12.5-corpus","cosmopedia-v2",split='test',num_proc=DEFAULT_NUM_PROC)
deepmind_math = load_dataset("chengjunyan1/smollm-12.5-corpus","deepmind-math-small",split='test',num_proc=DEFAULT_NUM_PROC)
stackoverflow = load_dataset("chengjunyan1/smollm-12.5-corpus","stackoverflow-clean",split='test',num_proc=DEFAULT_NUM_PROC)

cosmopedia_processed = cosmopedia.map(process_cosmopedia_v2, batched=True, batch_size=1000, num_proc=DEFAULT_NUM_PROC)
deepmind_math_processed = deepmind_math.map(process_deepmind_math_small, batched=True, batch_size=1000, num_proc=DEFAULT_NUM_PROC)
stackoverflow_processed = stackoverflow.rename_column('content', 'text')


Resolving data files:   0%|          | 0/69 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/69 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/237 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/69 [00:00<?, ?it/s]

{'blob_id',
 'int_score',
 'length_bytes',
 'path',
 'repo_name',
 'score',
 'token_counts'}

In [11]:
python_edu=python_edu.remove_columns(set(python_edu.column_names)-{'text'})
fineweb_edu_dedup=fineweb_edu_dedup.remove_columns(set(fineweb_edu_dedup.column_names)-{'text'})
open_web_math=open_web_math.remove_columns(set(open_web_math.column_names)-{'text'})
cosmopedia_processed=cosmopedia_processed.remove_columns(set(cosmopedia_processed.column_names)-{'text'})
deepmind_math_processed=deepmind_math_processed.remove_columns(set(deepmind_math_processed.column_names)-{'text'})
stackoverflow_processed=stackoverflow_processed.remove_columns(set(stackoverflow_processed.column_names)-{'text'})

In [18]:
python_edu=DatasetDict({'test':python_edu})
fineweb_edu_dedup=DatasetDict({'test':fineweb_edu_dedup})
open_web_math=DatasetDict({'test':open_web_math})
cosmopedia_processed=DatasetDict({'test':cosmopedia_processed})
deepmind_math_processed=DatasetDict({'test':deepmind_math_processed})
stackoverflow_processed=DatasetDict({'test':stackoverflow_processed})


In [20]:
python_edu.push_to_hub("chengjunyan1/smollm-12.5-test","python-edu")
fineweb_edu_dedup.push_to_hub("chengjunyan1/smollm-12.5-test","fineweb-edu-dedup")
open_web_math.push_to_hub("chengjunyan1/smollm-12.5-test","open-web-math")
cosmopedia_processed.push_to_hub("chengjunyan1/smollm-12.5-test","cosmopedia-v2")
deepmind_math_processed.push_to_hub("chengjunyan1/smollm-12.5-test","deepmind-math-small")
stackoverflow_processed.push_to_hub("chengjunyan1/smollm-12.5-test","stackoverflow-clean")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/70 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/95 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/95 [00:00<?, ?ba/s]

...

README.md:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chengjunyan1/smollm-12.5-test/commit/fc8ddcd756a70a88180e1c03a3e9cfb35178a2a1', commit_message='Upload dataset', commit_description='', oid='fc8ddcd756a70a88180e1c03a3e9cfb35178a2a1', pr_url=None, pr_revision=None, pr_num=None)